# 4. Sistemas Difusos para la Toma de Decisiones

- Autor: Rodrigo Salas, Dr. Ing.
- email: rodrigo.salas@uv.cl

En este notebook exploraremos el toolbox de SciKit-Fuzzy. Este documento está basado en la guía de usuario publicada:

https://pythonhosted.org/scikit-fuzzy/userguide/getting_started.html

Revisar la licencia de los códigos en:

https://pythonhosted.org/scikit-fuzzy/license.html

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import skfuzzy as fuzz
from skfuzzy import control as ctrl

%matplotlib inline

## 4.1. Principios de Control Difuso

La **lógica difusa** es una metodología basada en la idea de "grado de verdad", es decir, no solo se tiene verdadero o falso, sino que cualquier valor entre medio.

Una **variable difusa** tiene un valor nítido que adquiere algún número sobre un dominio predefinido (en términos de lógica difusa, llamado universo). El valor nítido es cómo pensamos en la variable usando las matemáticas normales. 

Una variable difusa también tiene varios términos que se usan para describir la variable. Los términos tomados juntos son el **conjunto difuso** que se puede usar para describir el *valor difuso* de una variable difusa. Estos términos suelen ser adjetivos como "pobre", "mediocre" y "bueno". Cada término tiene una función de membresía que define cómo un valor nítido se asigna al término en una escala de 0 a 1. 


Un **sistema de control difuso** vincula variables difusas utilizando un conjunto de reglas. Estas reglas son simplemente asignaciones que describen cómo una o más variables difusas se relacionan con otra. Estos se expresan en términos de una declaración IF-THEN; la parte IF se llama antecedente y la parte ENTONCES es la consecuente.

## 4.2. Ejemplo: El problema de las propinas

Se desea crear un controlador que estima la propina que debería entregarse a un restaurant.

**Antecedentes (entrada)**
- Servicio
    - Universo (rango de valores nítidos): ¿Qué tan bueno fue el servicio del mesero, en una escala de 1 a 10?
    - Conjuntos difusos (rango de valores difusos): pobre, aceptable, excelente
- Calidad de la Comida:
    - Universo: ¿Qué tan sabrosa fue la comida, en una escala de 1 a 10?
    - Conjunto Difuso: malo, regular, exquisito
    
**Consecuente (salida)**
- Propina:
    - Universo: ¿Cuánto debería ser la propina?, un porcentaje del valor total de la cuenta que va entre 0% y 25%
    - Conjunto Difuso: baja, media, alta

**Reglas:**

- SI el *servicio* fue pobre o la *calidad de la comida* fue mala, ENTONCES la propina será baja
- SI el *servicio* fue aceptable, ENTONCES la propina será media
- SI el *servicio* fue excelente la *calidad de la comida* fue exquisita ENTONCES la propina será alta.

**USO:**

- Al controlador le entrego:
    - El servicio fue 9.8
    - La calidad fue 6.5
- Recomendará que deje una propina del 20.2%

## 4.3. Sistema de Control Difuso 

1. Generar el universo de las variables

In [ ]:
x_qual = np.arange(0, 11, 1) # calidad [0,10]
x_serv = np.arange(0, 11, 1) # servicio [0,10]
x_tip  = np.arange(0, 26, 1) # propina [0,25]

2. Generara la funciones de membresías difusos

- Calidad:

In [ ]:
qual_lo = fuzz.trimf(x_qual, [0, 0, 5])
qual_md = fuzz.trimf(x_qual, [0, 5, 10])
qual_hi = fuzz.trimf(x_qual, [5, 10, 10])

- Servicio:

In [ ]:
serv_lo = fuzz.trimf(x_serv, [0, 0, 5])
serv_md = fuzz.trimf(x_serv, [0, 5, 10])
serv_hi = fuzz.trimf(x_serv, [5, 10, 10])

- Propina:

In [ ]:
tip_lo = fuzz.trimf(x_tip, [0, 0, 13])
tip_md = fuzz.trimf(x_tip, [0, 13, 25])
tip_hi = fuzz.trimf(x_tip, [13, 25, 25])

3. Visualización del universo de las funciones de membresías

In [ ]:
fig, (ax0, ax1, ax2) = plt.subplots(nrows=3, figsize=(8, 9))

ax0.plot(x_qual, qual_lo, 'b', linewidth=1.5, label='Pobre')
ax0.plot(x_qual, qual_md, 'g', linewidth=1.5, label='Aceptable')
ax0.plot(x_qual, qual_hi, 'r', linewidth=1.5, label='Excelente')
ax0.set_title('Calidad de la comida')
ax0.legend()

ax1.plot(x_serv, serv_lo, 'b', linewidth=1.5, label='Baja')
ax1.plot(x_serv, serv_md, 'g', linewidth=1.5, label='Media')
ax1.plot(x_serv, serv_hi, 'r', linewidth=1.5, label='Alta')
ax1.set_title('CAlidad del Servicio')
ax1.legend()

ax2.plot(x_tip, tip_lo, 'b', linewidth=1.5, label='Baja')
ax2.plot(x_tip, tip_md, 'g', linewidth=1.5, label='Media')
ax2.plot(x_tip, tip_hi, 'r', linewidth=1.5, label='Alta')
ax2.set_title('Porcentaje de propina')
ax2.legend()

for ax in (ax0, ax1, ax2):
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.get_xaxis().tick_bottom()
    ax.get_yaxis().tick_left()

plt.tight_layout()

4. Aplicación de las reglas
Obtención del consecuente dada la calidad de la comida de 6.5 y el servicio de 9.8


- Evaluación de la función de membresía en estos valores:

In [ ]:
qual_level_lo = fuzz.interp_membership(x_qual, qual_lo, 6.5)
qual_level_md = fuzz.interp_membership(x_qual, qual_md, 6.5)
qual_level_hi = fuzz.interp_membership(x_qual, qual_hi, 6.5)

serv_level_lo = fuzz.interp_membership(x_serv, serv_lo, 9.8)
serv_level_md = fuzz.interp_membership(x_serv, serv_md, 9.8)
serv_level_hi = fuzz.interp_membership(x_serv, serv_hi, 9.8)

**REGLA 1:** SI el *servicio* fue pobre o la *calidad de la comida* fue mala, ENTONCES la propina será baja

Operador MAX para el **OR**

In [ ]:
active_rule1 = np.fmax(qual_level_lo, serv_level_lo)

Obtención del consecuente utlizando la función MIN

In [ ]:
tip_activation_lo = np.fmin(active_rule1, tip_lo)  # removed entirely to 0

**REGLA 2:** SI el *servicio* fue aceptable, ENTONCES la propina será media

In [ ]:
# For rule 2 we connect acceptable service to medium tipping
tip_activation_md = np.fmin(serv_level_md, tip_md)

**REGLA 3:** SI el *servicio* fue excelente la *calidad de la comida* fue exquisita ENTONCES la propina será alta.

In [ ]:
active_rule3 = np.fmax(qual_level_hi, serv_level_hi)
tip_activation_hi = np.fmin(active_rule3, tip_hi)

5. Visualización del Consecuente

In [ ]:
tip0 = np.zeros_like(x_tip)

fig, ax0 = plt.subplots(figsize=(8, 3))

ax0.fill_between(x_tip, tip0, tip_activation_lo, facecolor='b', alpha=0.7)
ax0.plot(x_tip, tip_lo, 'b', linewidth=0.5, linestyle='--', label='baja')
ax0.fill_between(x_tip, tip0, tip_activation_md, facecolor='g', alpha=0.7)
ax0.plot(x_tip, tip_md, 'g', linewidth=0.5, linestyle='--', label='media')
ax0.fill_between(x_tip, tip0, tip_activation_hi, facecolor='r', alpha=0.7)
ax0.plot(x_tip, tip_hi, 'r', linewidth=0.5, linestyle='--', label='alta')
ax0.set_title('Activación de la función de membresía de la salida')
ax0.legend()

for ax in (ax0,):
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.get_xaxis().tick_bottom()
    ax.get_yaxis().tick_left()

plt.tight_layout()

6. Agregación de las Reglas

Con la activación de cada función de membresía de salida, todas las funciones de membresías tienen que ser combinadas.

Para agregar se utliza el oprador MAX

In [ ]:
aggregated = np.fmax(tip_activation_lo, np.fmax(tip_activation_md, tip_activation_hi))

7. Defuzzification

Finalmente se obtiene una respuesta dada por un valor real, el cual se obtiene con el método de los centroides.

In [ ]:
# Calculate defuzzified result
tip = fuzz.defuzz(x_tip, aggregated, 'centroid')
print("El porcentaje de valor total que se dará como propina es ", tip)

8. Visualización de la desfuzzificación

In [ ]:
fig, ax0 = plt.subplots(figsize=(8, 3))

tip_activation = fuzz.interp_membership(x_tip, aggregated, tip)  

ax0.plot(x_tip, tip_lo, 'b', linewidth=0.5, linestyle='--', )
ax0.plot(x_tip, tip_md, 'g', linewidth=0.5, linestyle='--')
ax0.plot(x_tip, tip_hi, 'r', linewidth=0.5, linestyle='--')
ax0.fill_between(x_tip, tip0, aggregated, facecolor='Orange', alpha=0.7)
ax0.plot([tip, tip], [0, tip_activation], 'k', linewidth=1.5, alpha=0.9)
ax0.set_title('Función de membresía agregada y resultado (linea)')

# Turn off top/right axes
for ax in (ax0,):
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.get_xaxis().tick_bottom()
    ax.get_yaxis().tick_left()

plt.tight_layout()

## 4.4. Creando un controlador usando SKFUZZY

1. Definición de las variables linguisticas

Definición del Universo

In [ ]:
quality = ctrl.Antecedent(np.arange(0, 11, 1), 'quality')
service = ctrl.Antecedent(np.arange(0, 11, 1), 'service')
tip = ctrl.Consequent(np.arange(0, 26, 1), 'tip')

Funciones de membresías de las variables lingüísticas creadas de forma automática

In [ ]:
quality.automf(3)
service.automf(3)

Funciones de membresías creadas de forma manual

In [ ]:
tip['low'] = fuzz.trimf(tip.universe, [0, 0, 13])
tip['medium'] = fuzz.trimf(tip.universe, [0, 13, 25])
tip['high'] = fuzz.trimf(tip.universe, [13, 25, 25])

2. Visualización

In [ ]:
quality['average'].view()

In [ ]:
service.view()

In [ ]:
tip.view()

3. Reglas difusas

**REGLA 1:** SI el *servicio* fue pobre o la *calidad de la comida* fue mala, ENTONCES la propina será baja

In [ ]:
rule1 = ctrl.Rule(quality['poor'] | service['poor'], tip['low'])

**REGLA 2:** SI el *servicio* fue aceptable, ENTONCES la propina será media

In [ ]:
rule2 = ctrl.Rule(service['average'], tip['medium'])

**REGLA 3:** SI el *servicio* fue excelente la *calidad de la comida* fue exquisita ENTONCES la propina será alta.

In [ ]:
rule3 = ctrl.Rule(service['good'] | quality['good'], tip['high'])

4. Creación del sistema de control

In [ ]:
tipping_ctrl = ctrl.ControlSystem([rule1, rule2, rule3])

5. Simulación del Sistema de Control

In [ ]:
tipping = ctrl.ControlSystemSimulation(tipping_ctrl)

Especificación de las entradas para que el simulador realice la computación

In [ ]:
tipping.input['quality'] = 6.5
tipping.input['service'] = 9.8

tipping.compute()

6. Propina sugerida

In [ ]:
propina = tipping.output['tip']
print("El porcentaje de valor total que se dará como propina es ", propina)

7. Visualización de la Salida

In [ ]:
tip.view(sim=tipping)